<a href="https://colab.research.google.com/github/tset-tset-tset/hoge/blob/master/JSUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## JSUT を落としてくる
!gdown --id 1f7bIQfwWdFOxeaYzs5Cw-HTcA8uwQ8qp -O jsut_ver1.1.zip
!unzip -q jsut_ver1.1.zip

Downloading...
From: https://drive.google.com/uc?id=1f7bIQfwWdFOxeaYzs5Cw-HTcA8uwQ8qp
To: /content/jsut_ver1.1.zip
2.69GB [00:12, 208MB/s]


In [2]:
## for mecab & mecab-ipadic-neologd
!sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!cd mecab-ipadic-neologd; sudo ./bin/install-mecab-ipadic-neologd -n -a -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.9).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 35 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional di

In [3]:
## for uconv
!sudo apt-get install icu-devtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
icu-devtools is already the newest version (60.2-3ubuntu3.1).
icu-devtools set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
%%bash

cd /content/jsut_ver1.1/

rm -f original.txt
rm -f metadata.csv
rm -f metadata_kana.csv
mkdir -p wavs

function convertjsut(){
    TARGET=$1
    cat ${TARGET}/transcript_utf8.txt >> original.txt
    cat ${TARGET}/transcript_utf8.txt | while IFS=: read id text; 
    do
        echo -n "${id}|" >> metadata_kana.csv
        cp ${TARGET}/wav/${id}.wav wavs/${id}.wav
        echo "${text}" | mecab -Odump -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd > /tmp/${id}.tmp
        cat /tmp/${id}.tmp | perl -lne '
            next if not m{\d+ (\S+) ([^,]+),(?:[^,]+,){4,6}([^,]+),};
            $add = ($3 eq "*" ? $1 : $3);
            if ($2 eq "助詞" or $2 eq "助動詞") {
                $f = 1; $o .= $add;
            } else {
                if ($f == 1) {
                    $r .= " $o"; $o = ""; $f = 0;
                }
                $o .= $add if $1 ne "BOS" and $1 ne "EOS";
            }
            END {
                $r = "$r $o";
                $r =~ s{^\s*(.*?)\s*$}{\1}g;
                $r =~ s{ ?、}{, }g;
                $r =~ s{ ?。}{.}g;
                print $r;
            }' >> metadata_kana.csv
        rm /tmp/${id}.tmp
    done
    ## 長音とくっついて 1 字になってしまうので最初に長音を変換する…
    cat metadata_kana.csv | perl -pe 's{ー}{-}g;' | uconv -f UTF-8 -t UTF-8 -x Latin | perl -pe 's{n'\''}{nn}g; s{i~}{}g;' > metadata.csv
}

convertjsut basic5000
convertjsut travel1000

## 2 カラム → 3 カラム
perl -F\\\| -i.org -alne 'print "$F[0]|$F[1]|$F[1]"' metadata.csv
perl -F\\\| -i.org -alne 'print "$F[0]|$F[1]|$F[1]"' metadata_kana.csv

echo [original.txt]
head original.txt -n5
tail original.txt -n5
echo
echo [metadata.csv]
head metadata.csv -n5
tail metadata.csv -n5
echo
echo [metadata_kana.csv]
head metadata_kana.csv -n5
tail metadata_kana.csv -n5

[original.txt]
BASIC5000_0001:水をマレーシアから買わなくてはならないのです。
BASIC5000_0002:木曜日、停戦会談は、何の進展もないまま終了しました。
BASIC5000_0003:上院議員は私がデータをゆがめたと告発した。
BASIC5000_0004:１週間して、そのニュースは本当になった。
BASIC5000_0005:血圧は、健康のパロメーターとして重要である。
TRAVEL1000_0996:チャンに後ほど、電話をさせましょうか。
TRAVEL1000_0997:私は、ギルフォイル会社の者です。
TRAVEL1000_0998:営利、非営利を問わず、禁止します。
TRAVEL1000_0999:荷物の預かり証を拝見できますか。
TRAVEL1000_1000:20ドル札を、くだいてください。

[metadata.csv]
BASIC5000_0001|mizuwo mare-shiakara kawanakuteha naranai nodesu.|mizuwo mare-shiakara kawanakuteha naranai nodesu.
BASIC5000_0002|mokuyoubi, teisenkaidanha, nanino shintenmo naimamashuuryoushimashita.|mokuyoubi, teisenkaidanha, nanino shintenmo naimamashuuryoushimashita.
BASIC5000_0003|jyouingiinha watashiga de-tawo yugametato kokuhatsushita.|jyouingiinha watashiga de-tawo yugametato kokuhatsushita.
BASIC5000_0004|isshuukanshite, sononyu-suha hontouninatta.|isshuukanshite, sononyu-suha hontouninatta.
BASIC5000_0005|ketsuatsuha, kenkouno parome-ta-toshite jyuuyoudearu.|ketsuatsuha, kenkouno parome-ta-t